In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import datetime
from datetime import timedelta 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## 0:Load Data

In [2]:
#Load the imputed df 
pd.set_option('display.max_columns',500)
crash = pd.read_csv('./data/Filtered_crash_records.csv', sep = ',')

####  Check Missingness

In [3]:
for col in crash.columns:
    miss_col = crash[col].isna().sum(axis=0)
    if miss_col > 0:
        print('{:<20s}{:>4d}'.format(col, miss_col))

CRASH_TYPE             1
LOC_DIR              395
RD_SURF                4
SURF_COND              9
LIGHT                  6
WEATHER                7
RD_DIV                18
RD_WRKZN              67


Records without `LOC_DIR` will be filtered as well. 

## 1: Combine TMC Link with Distance 

### Process TMC link and distance

In [4]:
#Load the TMC table
tmc_dist = pd.read_csv('./data/TMC_up_and_down_with_distance.csv')

In [5]:
# Load previous TMC df 
DATA_HOME = os.path.join('data', 'tmc_data')

TMC2013 = pd.read_csv(os.path.join(DATA_HOME, 'n2013.csv'))
TMC2014 = pd.read_csv(os.path.join(DATA_HOME, 'n2014.csv'))
TMC2015 = pd.read_csv(os.path.join(DATA_HOME, 'n2015.csv'))
TMC2016 = pd.read_csv(os.path.join(DATA_HOME, 'n2016.csv'))
TMC2017 = pd.read_csv(os.path.join(DATA_HOME, 'n2017.csv'))
TMC2018 = pd.read_csv(os.path.join(DATA_HOME, 'n2018.csv'))

In [6]:
tmc_dfs = pd.concat([TMC2013, TMC2014, TMC2015, TMC2016, TMC2017, TMC2018])

In [7]:
# Merge tables
def merge_tmc_dfs(tmc_dfs, tmc_dist):
    tmc_total = pd.merge(tmc_dfs, tmc_dist, how = 'left', left_on = 'TMC', right_on = 'tmc')
    tmc_total.drop(columns =['Unnamed: 0','tmc','tmc_dn3','tmc_up3', 'tmc_dn3.1', 'tmc_up3.1'], inplace = True)
    tmc_total.rename(columns ={'tmc_dn1.1':'tmc_dn1_miles', 'tmc_dn2.1':'tmc_dn2_miles', 
                'tmc_up1.1':'tmc_up1_miles', 'tmc_up2.1':'tmc_up2_miles'}, inplace = True)
    return tmc_total

In [8]:
tmc_total = merge_tmc_dfs(tmc_dfs, tmc_dist)
tmc_total['timestamp'] = pd.to_datetime(tmc_total['timestamp'])

In [9]:
tmc_total.sample(3)

,timestamp,TMC,tmc_dn1,tmc_dn2,tmc_up1,tmc_up2,tmc_miles,tmc_dn1_miles,tmc_dn2_miles,tmc_up1_miles,tmc_up2_miles
42869,2017-08-14 00:13:00,120-04736,120N04736,120-04735,120N04737,120-04737,0.388071,0.193976,0.004281,0.542501,0.538729
35033,2016-12-21 06:38:00,120N04729,120-13767,120N13767,120-04729,120-04730,0.480255,0.105961,0.089606,0.290355,0.137006
31522,2016-02-25 13:56:00,120+07442,120P07442,120+07443,120P04740,120+04740,0.540096,0.090041,0.297769,0.151087,0.335239


In [10]:
# add TMC downstream and upstream links
# add new speed column before crash, ranged between 1 and 15 minutes

def process_tmc_dfs(df):
    df['timestamp']      = df['timestamp'].astype(str)
    df['TMC_tstamp']     = df['TMC'] + " " +df['timestamp']
    df['TMC_dn1_tstamp'] = df['tmc_dn1'] + " " +df['timestamp']
    df['TMC_dn2_tstamp'] = df['tmc_dn2'] + " " +df['timestamp']
    df['TMC_up1_tstamp'] = df['tmc_up1'] + " " +df['timestamp']
    df['TMC_up2_tstamp'] = df['tmc_up2'] + " " +df['timestamp']
   
    minutes = range(1, 16)
    for minute in minutes:
        colname = 'pre_%d' % minute
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df[colname] = df['timestamp'].apply(lambda t: dt.datetime(t.year, t.month, t.day, t.hour, t.minute)\
                                           - timedelta(minutes = minute))
        
    colnames = ['TMC_%d', 'tmc_dn1_%d', 'tmc_dn2_%d', 'tmc_up1_%d', 'tmc_up2_%d']
    for minute in minutes:
        colname = 'pre_%d' % minute
        df[colname] = df[colname].astype(str)
        df[colnames[0] % minute] = df['TMC'] +" "+ df['pre_%d' % minute]
        df[colnames[1] % minute] = df['TMC'] +" "+ df['pre_%d' % minute]
        df[colnames[2] % minute] = df['TMC'] +" "+ df['pre_%d' % minute]
        df[colnames[3] % minute] = df['TMC'] +" "+ df['pre_%d' % minute]
        df[colnames[4] % minute] = df['TMC'] +" "+ df['pre_%d' % minute]    
    return df

In [11]:
tmc_total = process_tmc_dfs(tmc_total)

In [12]:
tmc_total.sample(3)

,timestamp,TMC,tmc_dn1,tmc_dn2,tmc_up1,tmc_up2,tmc_miles,tmc_dn1_miles,tmc_dn2_miles,tmc_up1_miles,tmc_up2_miles,TMC_tstamp,TMC_dn1_tstamp,TMC_dn2_tstamp,TMC_up1_tstamp,TMC_up2_tstamp,pre_1,pre_2,pre_3,pre_4,pre_5,pre_6,pre_7,pre_8,pre_9,pre_10,pre_11,pre_12,pre_13,pre_14,pre_15,TMC_1,tmc_dn1_1,tmc_dn2_1,tmc_up1_1,tmc_up2_1,TMC_2,tmc_dn1_2,tmc_dn2_2,tmc_up1_2,tmc_up2_2,TMC_3,tmc_dn1_3,tmc_dn2_3,tmc_up1_3,tmc_up2_3,TMC_4,tmc_dn1_4,tmc_dn2_4,tmc_up1_4,tmc_up2_4,TMC_5,tmc_dn1_5,tmc_dn2_5,tmc_up1_5,tmc_up2_5,TMC_6,tmc_dn1_6,tmc_dn2_6,tmc_up1_6,tmc_up2_6,TMC_7,tmc_dn1_7,tmc_dn2_7,tmc_up1_7,tmc_up2_7,TMC_8,tmc_dn1_8,tmc_dn2_8,tmc_up1_8,tmc_up2_8,TMC_9,tmc_dn1_9,tmc_dn2_9,tmc_up1_9,tmc_up2_9,TMC_10,tmc_dn1_10,tmc_dn2_10,tmc_up1_10,tmc_up2_10,TMC_11,tmc_dn1_11,tmc_dn2_11,tmc_up1_11,tmc_up2_11,TMC_12,tmc_dn1_12,tmc_dn2_12,tmc_up1_12,tmc_up2_12,TMC_13,tmc_dn1_13,tmc_dn2_13,tmc_up1_13,tmc_up2_13,TMC_14,tmc_dn1_14,tmc_dn2_14,tmc_up1_14,tmc_up2_14,TMC_15,tmc_dn1_15,tmc_dn2_15,tmc_up1_15,tmc_up2_15
24385,2015-03-15 02:35:00,120P04738,120+04739,120P04739,120+04738,120P04737,0.381707,0.221790,0.267140,0.280115,0.482554,120P04738 2015-03-15 02:35:00,120+04739 2015-03-15 02:35:00,120P04739 2015-03-15 02:35:00,120+04738 2015-03-15 02:35:00,120P04737 2015-03-15 02:35:00,2015-03-15 02:34:00,2015-03-15 02:33:00,2015-03-15 02:32:00,2015-03-15 02:31:00,2015-03-15 02:30:00,2015-03-15 02:29:00,2015-03-15 02:28:00,2015-03-15 02:27:00,2015-03-15 02:26:00,2015-03-15 02:25:00,2015-03-15 02:24:00,2015-03-15 02:23:00,2015-03-15 02:22:00,2015-03-15 02:21:00,2015-03-15 02:20:00,120P04738 2015-03-15 02:34:00,120P04738 2015-03-15 02:34:00,120P04738 2015-03-15 02:34:00,120P04738 2015-03-15 02:34:00,120P04738 2015-03-15 02:34:00,120P04738 2015-03-15 02:33:00,120P04738 2015-03-15 02:33:00,120P04738 2015-03-15 02:33:00,120P04738 2015-03-15 02:33:00,120P04738 2015-03-15 02:33:00,120P04738 2015-03-15 02:32:00,120P04738 2015-03-15 02:32:00,120P04738 2015-03-15 02:32:00,120P04738 2015-03-15 02:32:00,120P04738 2015-03-15 02:32:00,120P04738 2015-03-15 02:31:00,120P04738 2015-03-15 02:31:00,120P04738 2015-03-15 02:31:00,120P04738 2015-03-15 02:31:00,120P04738 2015-03-15 02:31:00,120P04738 2015-03-15 02:30:00,120P04738 2015-03-15 02:30:00,120P04738 2015-03-15 02:30:00,120P04738 2015-03-15 02:30:00,120P04738 2015-03-15 02:30:00,120P04738 2015-03-15 02:29:00,120P04738 2015-03-15 02:29:00,120P04738 2015-03-15 02:29:00,120P04738 2015-03-15 02:29:00,120P04738 2015-03-15 02:29:00,120P04738 2015-03-15 02:28:00,120P04738 2015-03-15 02:28:00,120P04738 2015-03-15 02:28:00,120P04738 2015-03-15 02:28:00,120P04738 2015-03-15 02:28:00,120P04738 2015-03-15 02:27:00,120P04738 2015-03-15 02:27:00,120P04738 2015-03-15 02:27:00,120P04738 2015-03-15 02:27:00,120P04738 2015-03-15 02:27:00,120P04738 2015-03-15 02:26:00,120P04738 2015-03-15 02:26:00,120P04738 2015-03-15 02:26:00,120P04738 2015-03-15 02:26:00,120P04738 2015-03-15 02:26:00,120P04738 2015-03-15 02:25:00,120P04738 2015-03-15 02:25:00,120P04738 2015-03-15 02:25:00,120P04738 2015-03-15 02:25:00,120P04738 2015-03-15 02:25:00,120P04738 2015-03-15 02:24:00,120P04738 2015-03-15 02:24:00,120P04738 2015-03-15 02:24:00,120P04738 2015-03-15 02:24:00,120P04738 2015-03-15 02:24:00,120P04738 2015-03-15 02:23:00,120P04738 2015-03-15 02:23:00,120P04738 2015-03-15 02:23:00,120P04738 2015-03-15 02:23:00,120P04738 2015-03-15 02:23:00,120P04738 2015-03-15 02:22:00,120P04738 2015-03-15 02:22:00,120P04738 2015-03-15 02:22:00,120P04738 2015-03-15 02:22:00,120P04738 2015-03-15 02:22:00,120P04738 2015-03-15 02:21:00,120P04738 2015-03-15 02:21:00,120P04738 2015-03-15 02:21:00,120P04738 2015-03-15 02:21:00,120P04738 2015-03-15 02:21:00,120P04738 2015-03-15 02:20:00,120P04738 2015-03-15 02:20:00,120P04738 2015-03-15 02:20:00,120P04738 2015-03-15 02:20:00,120P04738 2015-03-15 02:20:00
46911,2017-07-17 13:10:00,120+04737,120P04737,120+04738,120P04736,120+04736,0.644361,0.482554,0.280115,0.062811,0.036315,120+04737 2017-07-17 13:10:00,120P04737 2017-07-17 13:10:0

In [13]:
# Remove columns whose name start with 'pre_'
tmc_total = tmc_total[tmc_total.columns.drop(list(tmc_total.filter(regex='pre_')))]

## 2:Load INRIX speed data

### 2013 Data

In [14]:
DATA_HOME = os.path.join('data', 'inrix_data')

inrix2013 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2013.csv'))

### Drop irrelevant features

In [15]:
def inrix_pre_process(df):
    df.drop(columns=['Unnamed: 0','average_speed', 'reference_speed', \
                     'travel_time_seconds', 'confidence_score', 'cvalue'], inplace=True)
    df['tmc_tstamp'] = df['tmc_code']+' '+df['measurement_tstamp']
    df = df[['tmc_tstamp', 'speed']]
    df.drop_duplicates(subset=['tmc_tstamp'], inplace = True)
    return df

In [16]:
inrix2013 = inrix_pre_process(inrix2013)

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Add TMC + Timestamp Feature

In [17]:
def inrix_process(df, inrix):
    # Convert 'timestamp' to string 
    df['timestamp'] = df['timestamp'].astype(str)
    # merge crash link
    df = pd.merge(df, inrix, how='left', left_on='TMC_tstamp', right_on = 'tmc_tstamp')
    df.rename(columns = {'speed':'spd', 'tmc_tstamp':'tmc_tstamp_'}, inplace = True)
    # merge with the first downstream TMC link
    df = pd.merge(df, inrix, how='left', left_on='TMC_dn1_tstamp', right_on = 'tmc_tstamp')
    df.rename(columns = {'speed':'dn1_spd', 'tmc_tstamp':'tmc_tstamp_dn1'}, inplace = True)
    # merge with the second downstream TMC link
    df = pd.merge(df, inrix, how='left', left_on='TMC_dn2_tstamp', right_on = 'tmc_tstamp')
    df.rename(columns = {'speed':'dn2_spd', 'tmc_tstamp':'tmc_tstamp_dn2'}, inplace = True)
    # merge with the first upstream TMC link
    df = pd.merge(df, inrix, how='left', left_on='TMC_up1_tstamp', right_on = 'tmc_tstamp')
    df.rename(columns = {'speed':'up1_spd', 'tmc_tstamp':'tmc_tstamp_up1'}, inplace = True)
    # merge with the second upstream TMC link
    df = pd.merge(df, inrix, how='left', left_on='TMC_up2_tstamp', right_on = 'tmc_tstamp')
    df.rename(columns = {'speed':'up2_spd', 'tmc_tstamp':'tmc_tstamp_up1'}, inplace = True)
    return df

In [18]:
tmc2013 = tmc_total[tmc_total['timestamp'].dt.year == 2013]

In [19]:
tmc2013 = inrix_process(tmc2013, inrix2013) 

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
tmc2013.head()

,timestamp,TMC,tmc_dn1,tmc_dn2,tmc_up1,tmc_up2,tmc_miles,tmc_dn1_miles,tmc_dn2_miles,tmc_up1_miles,tmc_up2_miles,TMC_tstamp,TMC_dn1_tstamp,TMC_dn2_tstamp,TMC_up1_tstamp,TMC_up2_tstamp,TMC_1,tmc_dn1_1,tmc_dn2_1,tmc_up1_1,tmc_up2_1,TMC_2,tmc_dn1_2,tmc_dn2_2,tmc_up1_2,tmc_up2_2,TMC_3,tmc_dn1_3,tmc_dn2_3,tmc_up1_3,tmc_up2_3,TMC_4,tmc_dn1_4,tmc_dn2_4,tmc_up1_4,tmc_up2_4,TMC_5,tmc_dn1_5,tmc_dn2_5,tmc_up1_5,tmc_up2_5,TMC_6,tmc_dn1_6,tmc_dn2_6,tmc_up1_6,tmc_up2_6,TMC_7,tmc_dn1_7,tmc_dn2_7,tmc_up1_7,tmc_up2_7,TMC_8,tmc_dn1_8,tmc_dn2_8,tmc_up1_8,tmc_up2_8,TMC_9,tmc_dn1_9,tmc_dn2_9,tmc_up1_9,tmc_up2_9,TMC_10,tmc_dn1_10,tmc_dn2_10,tmc_up1_10,tmc_up2_10,TMC_11,tmc_dn1_11,tmc_dn2_11,tmc_up1_11,tmc_up2_11,TMC_12,tmc_dn1_12,tmc_dn2_12,tmc_up1_12,tmc_up2_12,TMC_13,tmc_dn1_13,tmc_dn2_13,tmc_up1_13,tmc_up2_13,TMC_14,tmc_dn1_14,tmc_dn2_14,tmc_up1_14,tmc_up2_14,TMC_15,tmc_dn1_15,tmc_dn2_15,tmc_up1_15,tmc_up2_15,tmc_tstamp_,spd,tmc_tstamp_dn1,dn1_spd,tmc_tstamp_dn2,dn2_spd,tmc_tstamp_up1,up1_spd,tmc_tstamp_up1,up2_spd
0,2013-06-23 18:01:00,120P13767,120+04729,120P04729,120P13767,120P13767,0.130332,0.136143,0.429524,0.130332,0.130332,120P13767 2013-06-23 18:01:00,120+04729 2013-06-23 18:01:00,120P04729 2013-06-23 18:01:00,120P13767 2013-06-23 18:01:00,120P13767 2013-06-23 18:01:00,120P13767 2013-06-23 18:00:00,120P13767 2013-06-23 18:00:00,120P13767 2013-06-23 18:00:00,120P13767 2013-06-23 18:00:00,120P13767 2013-06-23 18:00:00,120P13767 2013-06-23 17:59:00,120P13767 2013-06-23 17:59:00,120P13767 2013-06-23 17:59:00,120P13767 2013-06-23 17:59:00,120P13767 2013-06-23 17:59:00,120P13767 2013-06-23 17:58:00,120P13767 2013-06-23 17:58:00,120P13767 2013-06-23 17:58:00,120P13767 2013-06-23 17:58:00,120P13767 2013-06-23 17:58:00,120P13767 2013-06-23 17:57:00,120P13767 2013-06-23 17:57:00,120P13767 2013-06-23 17:57:00,120P13767 2013-06-23 17:57:00,120P13767 2013-06-23 17:57:00,120P13767 2013-06-23 17:56:00,120P13767 2013-06-23 17:56:00,120P13767 2013-06-23 17:56:00,120P13767 2013-06-23 17:56:00,120P13767 2013-06-23 17:56:00,120P13767 2013-06-23 17:55:00,120P13767 2013-06-23 17:55:00,120P13767 2013-06-23 17:55:00,120P13767 2013-06-23 17:55:00,120P13767 2013-06-23 17:55:00,120P13767 2013-06-23 17:54:00,120P13767 2013-06-23 17:54:00,120P13767 2013-06-23 17:54:00,120P13767 2013-06-23 17:54:00,120P13767 2013-06-23 17:54:00,120P13767 2013-06-23 17:53:00,120P13767 2013-06-23 17:53:00,120P13767 2013-06-23 17:53:00,120P13767 2013-06-23 17:53:00,120P13767 2013-06-23 17:53:00,120P13767 2013-06-23 17:52:00,120P13767 2013-06-23 17:52:00,120P13767 2013-06-23 17:52:00,120P13767 2013-06-23 17:52:00,120P13767 2013-06-23 17:52:00,120P13767 2013-06-23 17:51:00,120P13767 2013-06-23 17:51:00,120P13767 2013-06-23 17:51:00,120P13767 2013-06-23 17:51:00,120P13767 2013-06-23 17:51:00,120P13767 2013-06-23 17:50:00,120P13767 2013-06-23 17:50:00,120P13767 2013-06-23 17:50:00,120P13767 2013-06-23 17:50:00,120P13767 2013-06-23 17:50:00,120P13767 2013-06-23 17:49:00,120P13767 2013-06-23 17:49:00,120P13767 2013-06-23 17:49:00,120P13767 2013-06-23 17:49:00,120P13767 2013-06-23 17:49:00,120P13767 2013-06-23 17:48:00,120P13767 2013-06-23 17:48:00,120P13767 2013-06-23 17:48:00,120P13767 2013-06-23 17:48:00,120P13767 2013-06-23 17:48:00,120P13767 2013-06-23 17:47:00,120P13767 2013-06-23 17:47:00,120P13767 2013-06-23 17:47:00,120P13767 2013-06-23 17:47:00,120P13767 2013-06-23 17:47:00,120P13767 2013-06-23 17:46:00,120P13767 2013-06-23 17:46:00,120P13767 2013-06-23 17:46:00,120P13767 2013-06-23 17:46:00,120P13767 2013-06-23 17:46:00,120P13767 2013-06-23 18:01:00,30.0,120+04729 2013-06-23 18:01:00,39.0,120P04729 2013-06-23 18:01:00,48.0,120P13767 2013-06-23 18:01:00,30.0,120P13767 2013-06-23 18:01:00,30.0
1,2013-03-28 00:02:00,120P04734,120+04735,120P04735,120+04734,120P04733,0.207492,0.301162,0.297825,0.522380,0.164068,120P04734 2013-03-28 00:02:00,120+04735 2013-03-28 00:02:00,120P04735 2013-03-28 00:02:00,120+04734 2013-03-28 00:02:00,120P04733 2013-03-28 00:02:00,120P04734 2013-03-28 00:01:00

### Inrix pre-crash speed process

In [21]:
tmc2013 = tmc_total[tmc_total['timestamp'].dt.year == 2013]
tmc2014 = tmc_total[tmc_total['timestamp'].dt.year == 2014]
tmc2015 = tmc_total[tmc_total['timestamp'].dt.year == 2015]
tmc2016 = tmc_total[tmc_total['timestamp'].dt.year == 2016]
tmc2017 = tmc_total[tmc_total['timestamp'].dt.year == 2017]
tmc2018 = tmc_total[tmc_total['timestamp'].dt.year == 2018]

In [24]:

def inrix_process_precrash(df, inrix):
    minutes = range(1, 16)
    for minute in minutes:
        # Convert 'timestamp' to string 
        df['timestamp'] = df['timestamp'].astype(str)
        # merge crash link
        df = pd.merge(df, inrix, how='left', left_on='TMC_%d' % minute, right_on = 'tmc_tstamp')
        df.rename(columns = {'speed':'spd_%d' % minute, 'tmc_tstamp':'tmc_tstamp_%d' % minute}, inplace = True)
        # merge with the first downstream TMC link
        df = pd.merge(df, inrix, how='left', left_on='tmc_dn1_%d' % minute, right_on = 'tmc_tstamp')
        df.rename(columns = {'speed':'dn1_spd_%d' % minute, 'tmc_tstamp':'tmc_tstamp_dn1_%d' % minute}, inplace = True)
        # merge with the second downstream TMC link
        df = pd.merge(df, inrix, how='left', left_on='tmc_dn2_%d' % minute, right_on = 'tmc_tstamp')
        df.rename(columns = {'speed':'dn2_spd_%d' % minute, 'tmc_tstamp':'tmc_tstamp_dn2_%d' % minute}, inplace = True)
        # merge with the first upstream TMC link
        df = pd.merge(df, inrix, how='left', left_on='tmc_up1_%d' % minute, right_on = 'tmc_tstamp')
        df.rename(columns = {'speed':'up1_spd_%d' % minute, 'tmc_tstamp':'tmc_tstamp_up1_%d' % minute}, inplace = True)
        # merge with the second upstream TMC link
        df = pd.merge(df, inrix, how='left', left_on='tmc_up2_%d' % minute, right_on = 'tmc_tstamp')
        df.rename(columns = {'speed':'up2_spd_%d' % minute, 'tmc_tstamp':'tmc_tstamp_up2_%d' % minute}, inplace = True)
    return df

In [25]:
tmc2013 = inrix_process_precrash(tmc2013, inrix2013)

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
tmc2013.to_csv('./data/tmc_inrix_2013.csv')
del inrix2013

In [29]:
inrix2014 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2014.csv'))
inrix2014 = inrix_pre_process(inrix2014)
tmc2014 = inrix_process_precrash(tmc2014, inrix2014)
tmc2014.to_csv('./data/tmc_inrix_2014.csv')
del inrix2014

C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kkim1\miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
inrix2015 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2015.csv'))
inrix2015 = inrix_pre_process(inrix2015)
tmc2015 = inrix_process_precrash(tmc2015, inrix2015)
tmc2015.to_csv('./data/tmc_inrix_2015.csv')
del inrix2015

In [ ]:
inrix2016 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2016.csv'))
inrix2016 = inrix_pre_process(inrix2016)
tmc2016 = inrix_process_precrash(tmc2016, inrix2016)
tmc2016.to_csv('./data/tmc_inrix_2016.csv')
del inrix2016

In [ ]:
inrix2017 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2017.csv'))
inrix2017 = inrix_pre_process(inrix2017)
tmc2017 = inrix_process_precrash(tmc2017, inrix2017)
tmc2017.to_csv('./data/tmc_inrix_2017.csv')
del inrix2017

In [ ]:
inrix2018 = pd.read_csv(os.path.join(DATA_HOME, 'inrix_2018.csv'))
inrix2018 = inrix_pre_process(inrix2018)
tmc2018 = inrix_process_precrash(tmc2018, inrix2018)
tmc2018.to_csv('./data/tmc_inrix_2018.csv')
del inrix2018